In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class MouseDataset(Dataset):
    def __init__(self, root_dir):
        self.videos_dir = os.path.join(root_dir, 'videos')
        self.pupil_dir = os.path.join(root_dir, 'pupil_center')
        self.behavior_dir = os.path.join(root_dir, 'behavior')
        self.labels_dir = os.path.join(root_dir, 'labels')

        self.file_ids = sorted(
            [f.replace('.npy', '') for f in os.listdir(self.videos_dir) if f.endswith('.npy')],
            key=lambda x: int(x)
        )

    def __len__(self):
        return len(self.file_ids)

    def __getitem__(self, i):
        file_id = self.file_ids[i]

        video = np.load(os.path.join(self.videos_dir, f'{file_id}.npy'))
        pupil = np.load(os.path.join(self.pupil_dir, f'{file_id}.npy'))
        behavior = np.load(os.path.join(self.behavior_dir, f'{file_id}.npy'))
        label = np.load(os.path.join(self.labels_dir, f'{file_id}.npy'))

        video = video[np.newaxis, ...]
        
        return {
            'video': torch.from_numpy(video).float(),
            'pupil_center': torch.from_numpy(pupil).float().transpose(1,0),
            'behavior': torch.from_numpy(behavior).float().transpose(1,0),
            'labels': torch.from_numpy(label).float()
        }

In [2]:
train_set = MouseDataset('/kaggle/input/mousedatasetct/train/data')
validation_set = MouseDataset('/kaggle/input/mousedatasetct/validation/data')
test_set = MouseDataset('/kaggle/input/mousedatasetct/test/data')

In [23]:
train_set[0]['labels'].shape

torch.Size([227, 140])

# BASELINE

N.B: TRAIN_SET
* 0-41: indici associati a clip del natural_movie_one -> ogni 6 abbiamo una trial
* 42-216: indici associati a clip del natural_movie_three -> ogni 25, una trial

In [3]:
import numpy as np
import statistics

# funzione che prende in input train_set e restituisce la media dei dff di tutti i trial relativi ad una clip
def retrieve_dff_mean_among_trials_for_a_clip(movie_type: str , train_set: MouseDataset, clip: int):
    """
    movie_type:
        - "one"
        - "three"

    clip:
        - "one": da 1 a 6 (file da 0.npy a 5.npy), ritorno con clip=2: [1.npy, 7.npy, 13.npy, ... , 37.npy]
        - "three": da 1 a 25 (file da 42.npy a 66.npy), ritorno con clip=2: [43.npy, 68.npy, ... , 212.npy]
    
    """
    
    offset = 6 if movie_type=="one" else 25

    # Verifica che vengano passati parametri coerenti con il movie_type
    if movie_type == "one":
        # indici da 0 a 41 
        assert 1 <= clip <= 6, f"Per movie_type='one', clip deve essere tra 1 e 6, ricevuto: {clip}"
    elif movie_type == "three":
        # indici da 42 a 216 
        assert 1 <= clip <= 25, f"Per movie_type='three', clip deve essere tra 1 e 25, ricevuto: {clip}"
    else:
        raise ValueError(f"movie_type deve essere 'one' o 'three', ricevuto: {movie_type}")
    
    trials = []
    start_idx = clip-1 if movie_type == "one" else clip+41
    for i in range(7):
        trials.append(train_set[start_idx]['labels'].numpy())
        start_idx += offset
    
    # Media across trials: (7, 227, 140) → (227, 140)
    mean_across_trials = np.mean(trials, axis=0)
    return mean_across_trials

PASSIAMO AL TEST

In [4]:
print(len(test_set)) # 62 -> 12 per "one" (prime 6 per trial 8) e 50 per "three" (prime 25 per trial 8)


62


In [5]:
# ci recuperiamo i trial 8 e 9 del test per quanto riguarda clip 1 movie one
# N.B: una volta confrontiamo con 8 e un'altra con la 9
clip_1_type_one_test_trial8 = test_set[0]['labels'] 
clip_1_type_one_test_trial9 = test_set[6]['labels'] 

In [6]:
# ci recuperiamo i trial 8 e 9 del test per quanto riguarda clip 1 movie one
# N.B: una volta confrontiamo con 8 e un'altra con la 9
clip_1_type_three_test_trial8 = test_set[13]['labels'] 
clip_1_type_three_test_trial9 = test_set[38]['labels'] 

CORRELAZIONE

In [8]:
# Prima cella: calcolo della pearson correlation tra clip_1_type_one_train e test trials
import numpy as np

# Poi correlazione neurone-per-neurone
def compute_neuron_correlations(mean_training, test_trial):
    """Calcola correlazione neurone-per-neurone"""
    correlations = []

    print(mean_training.shape)
    for neuron in range(mean_training.shape[0]):
        # Ogni neurone ha 140 time points
        train_neuron = mean_training[neuron, :]  # shape (140,)
        test_neuron = test_trial[neuron, :]      # shape (140,)
        
        # Controlla varianza
        if np.var(train_neuron) > 1e-10 and np.var(test_neuron) > 1e-10:
            corr = np.corrcoef(train_neuron, test_neuron)[0, 1]
            if not np.isnan(corr) and np.isfinite(corr):
                correlations.append(corr)
    
    return {
        'mean_correlation': np.mean(correlations) if correlations else 0.0,
        'valid_neurons': len(correlations),
        'total_neurons': mean_training.shape[0]
    }

In [9]:
def pearson_correlation(predictions, labels):
             
    # Allineamento temporale per evaluation
    min_frames = 66
    labels_aligned = labels[..., -min_frames:]
    predictions_aligned = predictions[..., -min_frames:]
    
    # Reshape per correlazione: da (batch, neurons, time) a (batch*time, neurons)
    y_true = labels_aligned.transpose(1, 0)
    y_pred = predictions_aligned.transpose(1,0)
        
    correlations = []
        
    for neuron in range(y_true.shape[1]): 
        # per ogni neurone, recuperiamo predizione e label
        true_vals = y_true[:, neuron]
        pred_vals = y_pred[:, neuron]
        
        # Controlla varianza
        true_var = np.var(true_vals)
        pred_var = np.var(pred_vals)
        
        """
        filtriamo neuroni con attività praticamente costante (varianza quasi zero) prima di calcolare la correlazione. 
        Questo evita divisioni per zero nella formula della correlazione di Pearson.
        Neuroni "morti" o inattivi, comuni nei dataset neurali, avrebbero correlazioni matematicamente indefinite, 
        quindi vengono esclusi dal calcolo delle metriche. Solo neuroni con variabilità significativa 
        sia nei dati reali che nelle predizioni contribuiscono alla metrica finale di correlazione media, 
        garantendo risultati statisticamente validi.
        """
        if true_var > 1e-10 and pred_var > 1e-10:
            corr = np.corrcoef(true_vals, pred_vals)[0, 1]
            if not np.isnan(corr) and np.isfinite(corr):
                correlations.append(corr)
    
    mean_correlation = np.mean(correlations) if correlations else 0.0
        
    return {
        'eval_single_trial_correlation': mean_correlation,
    }

## APPROCCIO A "CAMPIONE"

Per il calcolo della baseline abbiamo proceduto nel modo seguente:
Il nostro train_set prevede 7 trials (da 0 a 7) sia di “natural_movie_one” che di “natural_movie_three” e per il processamento che abbiamo fatto relativamente al dataset (segmentazione) abbiamo ottenuto che il video di tipo “one”  consiste in 6 clip, mentre quello di tipo “three” in 25. Ciascuna clip ha shape (227,140).

In [10]:
# calcolo della pearson correlation tra clip_1_type_one_train e clip_1_type_one_test_trial8
clip_1_type_one_train = retrieve_dff_mean_among_trials_for_a_clip("one", train_set, 1)
result = pearson_correlation(clip_1_type_one_train, clip_1_type_one_test_trial8.numpy())
print(result) # 0.16688079896142594

{'eval_single_trial_correlation': 0.1503593076233057}


In [11]:
# calcolo della pearson correlation clip_1_type_one_train con clip_1_type_one_test_trial9
result = pearson_correlation(clip_1_type_one_train, clip_1_type_one_test_trial9.numpy())
print(result) # 0.15149332381948732

{'eval_single_trial_correlation': 0.15572641561251682}


In [12]:
# calcolo della pearson correlation clip_1_type_three_train con clip_1_type_one_test_trial8
clip_1_type_three_train = retrieve_dff_mean_among_trials_for_a_clip("three", train_set, 1)
result = pearson_correlation(clip_1_type_three_train, clip_1_type_three_test_trial8.numpy())
print(result) # 0.01626231821295263

{'eval_single_trial_correlation': 0.014238362417651177}


In [13]:
pearson_correlation# calcolo della pearson correlation clip_1_type_three_train con clip_1_type_one_test_trial9
result = pearson_correlation(clip_1_type_three_train, clip_1_type_three_test_trial9.numpy())
print(result) # 0.025573463105490166

{'eval_single_trial_correlation': 0.017345757218436755}


Dunque, abbiamo recuperato le trial della stessa clip nel test_set (nello specifico, le trial 8 e 9) e, infine, abbiamo calcolato la pearson correlation tra l’array NumPy ottenuto prima (valore medio, per ciascun neurone, della sua intensità nelle 7 trial, frame per frame, relativamente ad una clip selezionata) e la clip del trial 8 prima, e con la clip del trial 9 poi. 

## APPROCCIO CON MEDIA DELLE CORRELAZIONI SINGLE TRIAL

Dopo averci ragionato su, abbiamo ritenuto che calcolare la baseline rispetto a tutte le clip del dataset fosse un approccio più corretto sia perchè riesce a dare informazioni più significative rispetto all’approccio a campione, sia perchè l’addestramento del modello segue questa metrica (infatti, il metodo compute_metrics che viene richiamato dal Trainer di HuggingFace lo abbiamo implementato proprio per far restituire la media delle single trial correlation su tutti i campioni del test_set ). 
Conseguentemente, andando a fare l’evaluate() del modello sul test_set  si ottiene un valore che può essere confrontato direttamente con tale baseline. Nello specifico, abbiamo dapprima calcolato la correlazione tra le risposte del train_set e le clip del test_set relative alla trial 8 e poi abbiamo calcolato la correlazione tra le risposte del train_set e le clip del test_set relative alla trial 9. 


In [14]:
list_array_numpy = [] # avrà len=31
for movie_type in ["one","three"]:
    _range = 6 if movie_type == "one" else 25
    for i in range(_range):
        list_array_numpy.append(retrieve_dff_mean_among_trials_for_a_clip(movie_type, train_set, i+1))

print(list_array_numpy[0].shape)
# Convertiamo otteniamo un array 3D a partire dalla lista di array 2D
array_3d_train = np.stack(list_array_numpy, axis=0)
print(array_3d_train.shape)  # (31, 227, 140)

(227, 140)
(31, 227, 140)


In [15]:
# splittiamo i test set in 2 (trial 8 e 9):
# - 31 clip in toale: le prime 6 per "movie one" e le altre 25 (successive) per "movie three" <-> trial 8
# ...

trial_8_list = []
trial_9_list = []
for i in range(len(test_set)):
    if i < 31:
        trial_8_list.append(test_set[i]['labels'].numpy())
    else:
        trial_9_list.append(test_set[i]['labels'].numpy())

array_3d_trial8_test = np.stack(trial_8_list, axis=0)
array_3d_trial9_test = np.stack(trial_9_list, axis=0)
print(array_3d_trial8_test.shape)  # (31, 227, 140)
print(array_3d_trial9_test.shape)  # (31, 227, 140)

(31, 227, 140)
(31, 227, 140)


In [16]:
def compute_metrics(predictions, labels):  
    # Allineamento temporale
    min_frames = 66
    labels_aligned = labels[..., -min_frames:]
    predictions_aligned = predictions[..., -min_frames:]
    
    # Lista per memorizzare correlazioni per ogni esempio
    example_correlations = [] 
    
    # Loop per ogni esempio nel batch
    for example_idx in range(labels_aligned.shape[0]):
        # Dati per questo esempio specifico
        y_true_example = labels_aligned[example_idx].T  # (time, neurons)
        y_pred_example = predictions_aligned[example_idx].T  # (time, neurons)
        
        correlations = [] 
        

        for neuron in range(y_true_example.shape[1]):
            true_vals = y_true_example[:, neuron]
            pred_vals = y_pred_example[:, neuron]
            
            # Stesso filtering che avevi prima
            true_var = np.var(true_vals)
            pred_var = np.var(pred_vals)
            
            if true_var > 1e-10 and pred_var > 1e-10:
                corr = np.corrcoef(true_vals, pred_vals)[0, 1]
                if not np.isnan(corr) and np.isfinite(corr):
                    correlations.append(corr)
        
        # Correlazione media per questo esempio
        mean_corr_example = np.mean(correlations) if correlations else 0.0
        example_correlations.append(mean_corr_example) 
    
    # Statistiche aggregate
    overall_mean = np.mean(example_correlations) if example_correlations else 0.0
    
    return {
        'eval_average_single_trial_correlation': overall_mean,
        'eval_single_trial_std': np.std(example_correlations) if example_correlations else 0.0,
        'eval_num_examples': len(example_correlations),
    }

In [17]:
import statistics

results = [
    compute_metrics(array_3d_train, array_3d_trial8_test)["eval_average_single_trial_correlation"],
    compute_metrics(array_3d_train, array_3d_trial9_test)["eval_average_single_trial_correlation"]
]
print(results)

[0.007475139404624604, 0.039593989755850795]


In [18]:
print(f"Baseline among all data: {statistics.mean(results)} ")

Baseline among all data: 0.0235345645802377 
